In [1]:
from osgeo import ogr
from osgeo import gdal
import numpy as np
import pandas as pd
import geopandas as gpd
import critical_loads as cl
import nivapy

# Connect to db
engine = nivapy.da.connect()

# Critical Loads Workflow

This notebook implements a new workflow for the Critical Loads processing. There are three main components: vegetation, water and soils.

## 1. Vegetation

A new raster-based workflow was developed in [this notebook](http://nbviewer.jupyter.org/github/JamesSample/critical_loads/blob/master/notebooks/critical_loads_vegetation.ipynb). The code below loops over all the datasets to create vegetation exceedance grids (in mg-N/l) for the time periods of interest.

### 1.1. Upload new data to database

In [2]:
# ID for new series
ser_id = 26

# Create new series for this data period
df = pd.DataFrame({'dep_series_id':ser_id,
                   'name':'Middel 2012-2016',
                   'description':'Fordelt til BLR av NILU 2017 (Wenche Aas; old method)'},
                  index=[0,])

## Add to db
#df.to_sql('dep_series_definitions', con=engine, schema='resa2', 
#          if_exists='append', index=False)

In [3]:
# Read data
in_xlsx = (ur'K:\Avdeling\317 Klima- og miljømodellering\KAU\Focal Centre'
           ur'\Projects\2017 Tålegrenseprosjekt\Data\dep1216_til_NIVA_old method.xlsx')
df = pd.read_excel(in_xlsx, sheetname='16x16 NIVA', index_col=0)
df.index.name = 'blr'

# Map headings to db par IDs
par_dict = {'atnsss1216':4,  # Non-marine S
            'atnox1216':1,   # N (oks)
            'atnhx1216':2,   # N (red)
            'ats1216':3}     # Total S

# Get cols and rename
df = df[par_dict.keys()]
df.columns = [par_dict[i] for i in df.columns]
df.reset_index(inplace=True)

# Melt 
df = pd.melt(df, var_name='parameter_id', id_vars='blr')

# Add series ID
df['dep_series_id'] = ser_id

## Add to db
#df.to_sql('dep_blr_values', con=engine, schema='resa2', 
#          if_exists='append', index=False)

### 1.2. Read lookup table link vegetation classes to critical loads

In [4]:
# Read lookup table
in_xlsx = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation'
           r'\sat_veg_land_use_classes.xlsx')
df = pd.read_excel(in_xlsx, sheetname='EUNIS_tilGIS', index_col=0)

df = df[['CL_100smgN/m2/yr']].round(0).astype(int)

df.head()

,CL_100smgN/m2/yr
NORUTcode,
1,5
2,5
3,5
4,10
5,10


### 1.3. Reclassify

In [5]:
# Input national veg map
in_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_all.tif')

# Output geotiff for critical loads values
out_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
           r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Mask raster for land defined by BLR
mask_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\blr_land_mask.tif')

# Reclassify
cl.reclassify_raster(in_tif, mask_tif, out_tif, df, 'CL_100smgN/m2/yr', 255)

### 1.4. Get deposition data from database

**Note:** Remember to add the new `dep_series_id` to the query below, as well as a new name in `df.columns`.

In [6]:
# Get all dep values
sql = ("SELECT blr, dep_series_id as series, value as dep "
       "FROM resa2.dep_blr_values "
       "WHERE parameter_id IN (1, 2) "
       "AND dep_series_id IN (1, 2, 3, 4, 25, 26)")

df = pd.read_sql(sql, engine)

# Sum N_oks and N_red
df = df.groupby(['blr', 'series']).sum()

# Reshape and tidy
df = df.unstack()
df.columns = df.columns.get_level_values(1)
df.columns.name = ''
df.columns = ['Ndep78_82', 'Ndep92_96', 'Ndep97_01', 'Ndep02_06', 
              'Ndep07_11', 'Ndep12_16']

print np.nanmin(df.values), np.nanmax(df.values)

df.head()

69.8 2252.72


,Ndep78_82,Ndep92_96,Ndep97_01,Ndep02_06,Ndep07_11,Ndep12_16
blr,,,,,,
58006001,1700.00,1771.17,1576.91,NaN,1363.48,1402.38
58006002,1728.85,1247.05,1149.28,1192.63,1383.95,1290.48
58006003,1217.42,1171.20,1059.42,1157.39,1203.15,1128.59
58006004,1392.29,1310.32,1425.35,1302.15,1112.38,1095.33
58006005,1293.28,1302.73,1255.17,1037.57,1184.53,1147.69


### 1.5. Loop over datasets

In [7]:
%%time

# Path to raw BLR shapefile
in_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
          r'\Shapefiles\blrgrid_uten_grums_utm_z33n.shp')

# Snap raster
snap_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
            r'\GIS\Raster\sat_veg_30m_all.tif')

# Critical loads raster
cl_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
          r'\GIS\Raster\sat_veg_30m_cr_lds_div100.tif')

# Container for output
data_dict = {'series':[],
             'nor_area_km2':[],
             'ex_area_km2':[]}

# Loop over series
for ser in df.columns:    
    
    print 'Processing: %s' % ser
    print '    Building deposition shapefile...'
    
    # Get deposition
    dep_df = df[[ser]].dropna(how='any').round(0).astype(int).reset_index()

    # Rename cols to match shapefile
    dep_df.columns = ['BLR', ser]

    # Read shapefile
    blr_df = gpd.read_file(in_shp)

    # Join and tidy
    dep_df = blr_df.merge(dep_df, on='BLR')
    del dep_df['area_m2']
    
    # Write output shapefile
    dep_shp = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\GIS'
               r'\Shapefiles\dep_%s.shp' % ser)
    dep_df.to_file(dep_shp)
    
    # Convert shp to ras
    print '    Rasterising shapefile...'
    
    # Output BLR raster
    dep_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
               r'\GIS\Raster\dep_%s_30m.tif' % ser)

    cl.vec_to_ras(dep_shp, dep_tif, snap_tif, ser, -1, gdal.GDT_Int16)
    
    # Exceedance
    print '    Calculating exceedance...'
    
    # Read grids
    cl_grid, cl_ndv = cl.read_geotiff(cl_tif)
    dep_grid, dep_ndv = cl.read_geotiff(dep_tif)

    # Upcast to float32 for safe handling of negative values
    cl_grid = cl_grid.astype(np.float32)
    dep_grid = dep_grid.astype(np.float32)
   
    # Set ndv
    cl_grid[cl_grid==cl_ndv] = np.nan
    dep_grid[dep_grid==dep_ndv] = np.nan

    # Get total area of non-NaN from dep grid
    nor_area = np.count_nonzero(~np.isnan(dep_grid))*30.*30./1.E6

    # Apply scaling factor to CLs
    cl_grid = cl_grid*100.

    # Exceedance
    ex_grid = dep_grid - cl_grid
    del dep_grid, cl_grid  
    
    # Get total area exceeded
    ex_area = np.count_nonzero(ex_grid > 0)*30.*30./1.E6

    # Set <0 to 0
    ex_grid[ex_grid<0] = 0
    
    # Reset ndv
    ex_grid[np.isnan(ex_grid)] = -1

    # Downcast to int16 to save space
    ex_grid = ex_grid.round(0).astype(np.int16)
    
    # Append results
    data_dict['series'].append(ser)
    data_dict['nor_area_km2'].append(nor_area)
    data_dict['ex_area_km2'].append(ex_area)
    
    # Write output
    print '    Saving exceedance grid...'
    ex_tif = (r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads'
              r'\GIS\Raster\exceed_%s_30m.tif' % ser)
    
    cl.write_geotiff(ex_grid, ex_tif, snap_tif, -1, gdal.GDT_Int16)
    
    print '    Done.'

# Build output df
ex_df = pd.DataFrame(data_dict)
ex_df['ex_pct'] = 100 * ex_df['ex_area_km2'] / ex_df['nor_area_km2']
ex_df.index = ex_df['series']
del ex_df['series']
ex_df = ex_df.round(0).astype(int)

# Save
out_csv = r'C:\Data\James_Work\Staff\Kari_A\Critical_Loads\vegetation\nor_prop_exceed.csv'
ex_df.to_csv(out_csv)

print 'Finished.'

Processing: Ndep78_82
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...

C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:78: RuntimeWarning: invalid value encountered in greater
C:\Data\WinPython-64bit-2.7.10.3\python-2.7.10.amd64\lib\site-packages\ipykernel\__main__.py:81: RuntimeWarning: invalid value encountered in less



    Saving exceedance grid...
    Done.
Processing: Ndep92_96
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep97_01
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep02_06
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep07_11
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Processing: Ndep12_16
    Building deposition shapefile...
    Rasterising shapefile...
    Calculating exceedance...
    Saving exceedance grid...
    Done.
Finished.
Wall time: 14min 10s


### 1.6. Print national exceedance summary for vegetation

In [8]:
# Exceedance summary
ex_df

,ex_area_km2,nor_area_km2,ex_pct
series,,,
Ndep78_82,95122,320584,30
Ndep92_96,71470,320584,22
Ndep97_01,69485,320584,22
Ndep02_06,71065,320390,22
Ndep07_11,68475,320584,21
Ndep12_16,96409,320584,30
